In [1]:
# импортируем библиотеки pandas, numpy
import pandas as pd
import numpy as np
# импортируем функции train_test_split() и display()
from sklearn.model_selection import train_test_split
from IPython.display import display
# импортируем класс StandardScaler, 
# выполняющий стандартизацию
from sklearn.preprocessing import StandardScaler
# импортируем класс LogisticRegression
from sklearn.linear_model import LogisticRegression
# импортируем класс DecisionTreeRegressor
from sklearn.tree import DecisionTreeRegressor

In [2]:
# записываем CSV-файл в объект DataFrame
data = pd.read_csv('Data/StateFarm.csv', sep=';') 

In [3]:
# сформируем игрушечные массив меток 
# и массив признаков
var_lst = ['Income', 'Monthly Premium Auto', 
           'Customer Lifetime Value']
toy_data = data[var_lst].head(10)
toy_labels = toy_data.pop('Customer Lifetime Value')

In [4]:
# смотрим массив признаков
toy_data

,Income,Monthly Premium Auto
0,65999,237
1,0,65
2,54500,63
3,37260,62
4,68987,71
5,42305,117
6,65706,91
7,0,90
8,53243,66
9,0,70


In [5]:
# смотрим массив меток
toy_labels

0    18975.456110
1     4715.321344
2     5018.885233
3     4932.916345
4     5744.229745
5    13891.735670
6     7380.976717
7     3090.034104
8     2521.633095
9     2652.061785
Name: Customer Lifetime Value, dtype: float64

In [6]:
# получаем индексы наблюдений исходной выборки
sample_indices = np.arange(toy_data.shape[0])

# задаем стартовое значение генератора
# псевдослучайных чисел
rng = np.random.RandomState(42)

# получаем индексы наблюдений бутстреп-выборки:
# sample_indices - используем индексы исходной выборки
# size - размер тот же, что у исходной выборки
# replace=True - отбор с возвращением
bootstrap_indices = rng.choice(sample_indices,
                               size=sample_indices.shape[0],
                               replace=True)

In [7]:
# получаем бутстреп-выборку
toy_data_boot = toy_data.iloc[bootstrap_indices]
toy_labels_boot = toy_labels.iloc[bootstrap_indices]

display(toy_data_boot)
display(toy_labels_boot)

,Income,Monthly Premium Auto
6,65706,91
3,37260,62
7,0,90
4,68987,71
6,65706,91
9,0,70
2,54500,63
6,65706,91
7,0,90
4,68987,71


6    7380.976717
3    4932.916345
7    3090.034104
4    5744.229745
6    7380.976717
9    2652.061785
2    5018.885233
6    7380.976717
7    3090.034104
4    5744.229745
Name: Customer Lifetime Value, dtype: float64

In [8]:
# получаем out-of-bag выборку
toy_data_out_boot = toy_data[~toy_data.index.isin(
    toy_data_boot.index)]
toy_labels_out_boot = toy_labels[~toy_labels.index.isin(
    toy_data_boot.index)]

display(toy_data_out_boot)
display(toy_labels_out_boot)

,Income,Monthly Premium Auto
0,65999,237
1,0,65
5,42305,117
8,53243,66


0    18975.456110
1     4715.321344
5    13891.735670
8     2521.633095
Name: Customer Lifetime Value, dtype: float64

In [9]:
# пишем функцию, возвращающую бутстреп-выборки
# и out-of-bag выборки
def generate_bootstrap(rng, X, y, verbose=True):
    # получаем индексы наблюдений исходной выборки
    sample_indices = np.arange(X.shape[0])
    # получаем индексы наблюдений бутстреп-выборки, 
    # бутстреп-выборка имеет тот же размер, 
    # что и исходная, отбор с возвращением
    bootstrap_indices = rng.choice(sample_indices,
                                   size=sample_indices.shape[0],
                                   replace=True)
    X_boot = X.iloc[bootstrap_indices]
    y_boot = y.iloc[bootstrap_indices]
    X_out_boot = X[~X.index.isin(X_boot.index)]
    y_out_boot = y[~y.index.isin(X_boot.index)]
    if verbose:
        print(f"{i}-итерация")
        print(f"индексы в бутстреп-выборке: {X_boot.index.tolist()}")
        print(f"индексы в out-of-bag выборке: {X_out_boot.index.tolist()}\n")
    # возвращаем выборки
    return X_boot, y_boot, X_out_boot, y_out_boot

# создаем контейнер для генератора
# псевдослучайных чисел
rng = np.random.RandomState(42)
    
# создаем экземпляр класса DecisionTreeRegressor
tree = DecisionTreeRegressor(random_state=42)

# создаем пустой список, в который
# будем записывать значения R2
test_score_lst = []
# на каждой итерации...
for i in range(1, 4):
    # формируем бустреп-выборку и out-of-bag выборку
    X_boot, y_boot, X_out_boot, y_out_boot = generate_bootstrap(
        rng, toy_data, toy_labels)
    # обучаем на бутстреп-выборке
    tree.fit(X_boot, y_boot)
    # записываем значение R2
    test_score = tree.score(X_out_boot, y_out_boot)
    # кладем значение R2 в список
    test_score_lst.append(test_score)

1-итерация
индексы в бутстреп-выборке: [6, 3, 7, 4, 6, 9, 2, 6, 7, 4]
индексы в out-of-bag выборке: [0, 1, 5, 8]

2-итерация
индексы в бутстреп-выборке: [3, 7, 7, 2, 5, 4, 1, 7, 5, 1]
индексы в out-of-bag выборке: [0, 6, 8, 9]

3-итерация
индексы в бутстреп-выборке: [4, 0, 9, 5, 8, 0, 9, 2, 6, 3]
индексы в out-of-bag выборке: [1, 7]



In [10]:
# смотрим список
print(test_score_lst)

[-0.043216359884440836, 0.6652528178872499, -2.368359678979963]


In [11]:
# получаем среднее значение R2
mean_r2 = sum(test_score_lst) / len(test_score_lst)
print("Среднее значение R2: %.3f" % mean_r2)

Среднее значение R2: -0.582


In [12]:
# среднее значение в списке можно вычислить 
# с помощью mean() или fmean() пакета statistics
import statistics
mean_r2 = statistics.fmean(test_score_lst)
print("Среднее значение R2: %.3f" % mean_r2)

Среднее значение R2: -0.582


In [13]:
# разбиваем данные на обучающие и тестовые: получаем обучающий
# массив признаков, тестовый массив признаков, обучающий массив
# меток, тестовый массив меток
X_train, X_test, y_train, y_test = train_test_split(
    data.drop('Response', axis=1), 
    data['Response'], 
    test_size=0.3,
    stratify=data['Response'],
    random_state=42)

In [14]:
# создаем контейнер для генератора
# псевдослучайных чисел
rng = np.random.RandomState(42)

# создаем экземпляр класса StandardScaler
standardscaler = StandardScaler()

# создаем экземпляр класса LogisticRegression
logreg = LogisticRegression(solver='lbfgs', max_iter=200)

# создаем пустой список, в который будем
# записывать значения правильности
test_score_lst = []

# на каждой итерации...
for i in range(1000):
    # формируем бустреп-выборку и out-of-bag выборку
    X_boot, y_boot, X_out_boot, y_out_boot = generate_bootstrap(
        rng, X_train, y_train, verbose=False)
    # выполняем стандартизацию
    standardscaler.fit(X_boot)
    X_boot_scaled = standardscaler.transform(X_boot)
    X_out_boot_scaled = standardscaler.transform(X_out_boot)
    # обучаем на бутстреп-выборке
    logreg.fit(X_boot_scaled, y_boot)
    # записываем значение правильности
    test_score = logreg.score(
        X_out_boot_scaled, y_out_boot)
    # кладем значение правильности в список
    test_score_lst.append(test_score)

In [15]:
# получаем среднее значение правильности
mean_acc = statistics.fmean(test_score_lst)
print("Среднее значение правильности: %.3f" % mean_acc)

Среднее значение правильности: 0.900


In [16]:
# импортируем необходимые данные
from mlxtend.data import boston_housing_data

In [17]:
# создаем массив признаков и массив значений зависимой переменной
X, y = boston_housing_data()
# создаем обучающие и тестовые массивы признаков
# и значений зависимой переменной
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y,
                                                    test_size=0.3,
                                                    random_state=123)

In [18]:
# пишем функцию, генерирующую бутстреп-выборки
def _draw_bootstrap_sample(rng, X, y):
    # получаем индексы наблюдений исходной выборки
    sample_indices = np.arange(X.shape[0])
    # получаем индексы наблюдений бутстреп-выборки, бутстреп-выборка 
    # имеет тот же размер, что и исходная, отбор с возвращением
    bootstrap_indices = rng.choice(sample_indices,
                                   size=sample_indices.shape[0],
                                   replace=True)
    # формируем бутстреп-выборку
    return X[bootstrap_indices], y[bootstrap_indices]

# пишем функцию, вычисляющую усредненное ожидаемое значение функции
# потерь, усредненное смещение, усредненную дисперсию
def bias_variance_decomp(estimator, X_train, y_train, X_test, y_test,
                         num_rounds=200, random_seed=None):
    """
    Автор: Sebastian Raschka
    https://github.com/rasbt

    Параметры
    ----------
    estimator: регрессор.
    X_train: обучающий набор для извлечения бутстреп-выборок.
    y_train: обучающий массив значений зависимой переменной.
    X_test: тестовый набор для вычисления среднеквадратичной функции
        потерь, смещения и дисперсии.
    y_test: тестовый массив значений зависимой переменной.
    num_rounds : количество итераций бутстрепа (целочисленное 
        значение, по умолчанию 200).
    random_seed : стартовое значение генератора псевдослучайных
        чисел для создания бутстреп-выборок (целочисленное 
        значение, по умолчанию None).
    
    Возвращает
    ----------
    avg_expected_loss, avg_bias, avg_var : усредненное
        ожидаемое значение функции потерь, усредненное смещение,
        усредненную дисперсию (все значения с плавающей точкой),
        усреднение происходит по наблюдениям тестового набора.
    """
    
    # создаем контейнер для генератора
    # псевдослучайных чисел
    rng = np.random.RandomState(random_seed)

    # создаем массив из нулей, количество строк равно num_rounds, 
    # а количество столбцов определяется количеством наблюдений 
    # тестового набора, в него мы будем записывать прогнозы
    all_pred = np.zeros((num_rounds, y_test.shape[0]), dtype=int)

    # на каждой итерации...
    for i in range(num_rounds):
        # формируем бутстреп-выборку на основе обучающего набора
        X_boot, y_boot = _draw_bootstrap_sample(rng, X_train, y_train)
        # обучаем регрессор на бутстреп-выборке и
        # выдаем прогнозы для тестового набора
        pred = estimator.fit(X_boot, y_boot).predict(X_test)
        # записываем прогнозы в массив all_pred
        all_pred[i] = pred

    # вычисляем усредненное ожидаемое значение функции потерь, 
    # в нашем случае среднеквадратичную ошибку, усредненную 
    # по среднеквадратичным ошибкам, полученным на каждой
    # итерации (количество итераций задается num_rounds)
    avg_expected_loss = np.apply_along_axis(
        lambda x:
        ((x - y_test) ** 2).mean(),
        axis=1,
        arr=all_pred).mean()

    # вычисляем усредненный прогноз (берем среднее 
    # по оси 0 массива all_pred)
    main_predictions = np.mean(all_pred, axis=0)
    
    # вычисляем усредненное смещение, делим сумму квадратов 
    # разностей между усредненными прогнозами и фактическими 
    # значениями зависимой переменной в тестовом наборе на
    # количество наблюдений тестового набора
    avg_bias = np.sum((main_predictions - y_test) ** 2) / y_test.size
    # вычисляем усредненную дисперсию,  делим сумму квадратов
    # разностей между усредненными прогнозами и прогнозами 
    # для тестового набора на количество прогнозов
    avg_var = np.sum((main_predictions - all_pred) ** 2) / all_pred.size
    
    return avg_expected_loss, avg_bias, avg_var

In [19]:
# создаем дерево максимальной глубины
tree = DecisionTreeRegressor(random_state=123)

# вычисляем усредненное ожидаемое значение функции потерь,
# усредненное смещение, усредненную дисперсию
avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
    tree, X_train, y_train, X_test, y_test,
    random_seed=123)

# печатаем результаты
ttl = "Усредненное ожидаемое значение функции потерь: %.3f"
print(ttl % avg_expected_loss)
print("Усредненное смещение: %.3f" % avg_bias)
print("Усредненная дисперсия: %.3f" % avg_var)

Усредненное ожидаемое значение функции потерь: 31.756
Усредненное смещение: 13.856
Усредненная дисперсия: 17.900


In [20]:
# создаем дерево глубиной 1
tree2 = DecisionTreeRegressor(random_state=123, max_depth=1)

# вычисляем усредненное ожидаемое значение функции потерь,
# усредненное смещение, усредненную дисперсию
avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
    tree2, X_train, y_train, X_test, y_test, 
    random_seed=123)

# печатаем результаты
print(ttl % avg_expected_loss)
print("Усредненное смещение: %.3f" % avg_bias)
print("Усредненная дисперсия: %.3f" % avg_var)

Усредненное ожидаемое значение функции потерь: 53.199
Усредненное смещение: 41.990
Усредненная дисперсия: 11.209


In [21]:
# создаем игрушечные обучающие и тестовые массивы 
# признаков и значений зависимой переменной

X_train = np.array([[29.1, 19000.28, 15], 
                    [67.3, 48800.81, 45], 
                    [77.9, 89800.55, 188]])

X_test = np.array([[11.9, 89900.28, 199], 
                   [37.8, 10600.82, 95], 
                   [77.2, 99700.22, 87]])

y_train = np.array([22.6, 89.5, 17.3])
y_test = np.array([12.4, 96.9, 107.9])

In [22]:
# создаем контейнер для генератора
# псевдослучайных чисел
rng = np.random.RandomState(123)

In [23]:
# создаем массив из нулей, количество строк равно количеству итераций, 
# а количество столбцов определяется количеством наблюдений тестового 
# набора, в этот массив мы будем записывать прогнозы
all_pred = np.zeros((3, y_test.shape[0]), dtype=int)
all_pred

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]])

In [24]:
# на каждой итерации...
for i in range(3):
    # формируем бутстреп-выборку на основе обучающего набора
    X_boot, y_boot = _draw_bootstrap_sample(rng, X_train, y_train)
    # обучаем регрессор на бутстреп-выборке и 
    # выдаем прогнозы для тестового набора
    pred = tree.fit(X_boot, y_boot).predict(X_test)
    # записываем прогнозы в массив all_pred
    all_pred[i] = pred

In [25]:
# смотрим массив с прогнозами
all_pred

array([[17, 89, 89],
       [17, 22, 22],
       [17, 89, 89]])

In [26]:
# смотрим тестовый массив значений 
# зависимой переменной
y_test

array([ 12.4,  96.9, 107.9])

In [27]:
# вычислим среднеквадратичную ошибку по прогнозам,
# полученным на первой итерации
mse_first_iter = (((17 - 12.4)**2) + ((89 - 96.9)**2) + 
                  ((89 - 107.9)**2)) / 3
mse_first_iter

146.92666666666676

In [28]:
# вычислим среднеквадратичные ошибки по прогнозам,
# полученным на каждой итерации
mse = np.apply_along_axis(
    lambda x:
    ((x - y_test)**2).mean(),
    axis=1,
    arr=all_pred)
mse

array([ 146.92666667, 4336.66      ,  146.92666667])

In [29]:
# вычисляем усредненное ожидаемое значение функции потерь, 
# в нашем случае среднеквадратичную ошибку, усредненную по 
# среднеквадратичным ошибкам, полученным на каждой итерации
avg_expected_loss = np.apply_along_axis(
    lambda x:
    ((x - y_test)**2).mean(),
    axis=1,
    arr=all_pred).mean()
avg_expected_loss

1543.504444444445

In [30]:
# вычисляем усредненный прогноз по каждому наблюдению 
# (берем среднее по оси 0 массива all_pred)
main_predictions = np.mean(all_pred, axis=0)
main_predictions

array([17.        , 66.66666667, 66.66666667])

In [31]:
# вычисляем усредненное смещение, делим сумму квадратов разностей 
# между усредненными прогнозами и фактическими значениями 
# зависимой переменной в тестовом наборе на
# количество наблюдений тестового набора
avg_bias = np.sum((main_predictions - y_test)**2) / y_test.size
avg_bias

878.4674074074074

In [32]:
# вычисляем усредненную дисперсию, делим сумму квадратов разностей 
# между усредненными прогнозами и прогнозами для тестового набора
# на количество прогнозов
avg_var = np.sum((main_predictions - all_pred)**2) / all_pred.size
avg_var

665.037037037037

In [33]:
# давайте проверим, совпадают ли результаты, вычисленные вручную, с 
# результатами, вычисленными с помощью функции bias_variance_decomp()

# вычисляем усредненное ожидаемое значение функции потерь,
# усредненное смещение, усредненную дисперсию
avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
    tree, X_train, y_train, X_test, y_test, num_rounds=3,
    random_seed=123)

# печатаем результаты
print(ttl % avg_expected_loss)
print("Усредненное смещение: %.3f" % avg_bias)
print("Усредненная дисперсия: %.3f" % avg_var)

Усредненное ожидаемое значение функции потерь: 1543.504
Усредненное смещение: 878.467
Усредненная дисперсия: 665.037
